# 1. Imports

In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:
%pwd

'/Users/apple/MDS/block6/525/DSCI_525_Group26/notebooks'

# 2. Downloading data

In [3]:
%cd /Users/apple/MDS/block6/525/DSCI_525_Group26/notebooks

/Users/apple/MDS/block6/525/DSCI_525_Group26/notebooks


In [4]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [5]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

## 2.1 Downloading zip file

In [8]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 6.45 s, sys: 5.4 s, total: 11.9 s
Wall time: 1min 16s


## 2.2 Extracting the zip file

In [9]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 31 s, sys: 7.2 s, total: 38.2 s
Wall time: 1min 46s


In [11]:
%ls -ltr figsharerainfall/

total 12447120
-rw-r--r--   1 apple  staff  814041183 Mar 29 15:08 data.zip
-rw-r--r--   1 apple  staff   95376895 Mar 29 15:08 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   94960113 Mar 29 15:08 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   82474546 Mar 29 15:08 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  127613760 Mar 29 15:08 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  232118894 Mar 29 15:08 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  330360682 Mar 29 15:08 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  254009247 Mar 29 15:08 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  235661418 Mar 29 15:09 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  294260911 Mar 29 15:09 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  295768615 Mar 29 15:09 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  328852379 Mar 29 1

> Data Download Comparison

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Adam Morphy | MacOS Big Sur | 8GB | 1.8 GHz Dual-Core Intel Core i5 | Yes | |
| Mukund Iyer | MacOS Monterey | 8GB | 1.4 GHz Quad-Core Intel Core i5 | Yes | |
| Julien Gordon | Ubuntu 20.04.4 LTS | 16GB | AMD® Ryzen 7 5800h with radeon graphics | Yes | |
| Shiva Shankar Jena | MacOS Catalina 10.15.7 | 4GB | 1.4 GHz Dual-Core Intel Core i5 | Yes | 1m 16s |

# 3. Combining data CSVs

In [2]:
%%time

use_cols = ["rain", "lat_min", "lat_max", "Ion_min", "Ion_max", "rain (mm/day)", "model"]
files = glob.glob('figsharerainfall/*.csv')
excluded_files = ["figsharerainfall/observed_daily_rainfall_SYD.csv"]
df = pd.concat(
    (
        pd.read_csv(file, index_col=0)
        .assign(model=re.findall(r'\/(.*?)_', file)[0])
        for file in files
        if file not in excluded_files
        
    )
)
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 10min 2s, sys: 1min 1s, total: 11min 4s
Wall time: 12min 58s


> Combining Data Comparison

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Adam Morphy | MacOS Big Sur | 8GB | 1.8 GHz Dual-Core Intel Core i5 | Yes | |
| Mukund Iyer | MacOS Monterey | 8GB | 1.4 GHz Quad-Core Intel Core i5 | Yes | |
| Julien Gordon | Ubuntu 20.04.4 LTS | 16GB | AMD® Ryzen 7 5800h with radeon graphics | Yes | |
| Shiva Shankar Jena | MacOS Catalina 10.15.7 | 4GB | 1.4 GHz Dual-Core Intel Core i5 | Yes | 12m 58s |

# 4. Load the combined CSV to memory and perform a simple EDA

## 4.1 Loading the whole data and performing EDA

In [3]:
%%time

df_combined = pd.read_csv(
    "figsharerainfall/combined_data.csv", 
    index_col=0,
    parse_dates=True 
)

CPU times: user 1min 44s, sys: 47.8 s, total: 2min 32s
Wall time: 3min 53s


In [4]:
%%time

# EDA
df_combined.model.nunique()

CPU times: user 4.27 s, sys: 1.49 s, total: 5.76 s
Wall time: 6.6 s


27

In [5]:
%%time

df_combined.model.value_counts()

CPU times: user 4.91 s, sys: 528 ms, total: 5.44 s
Wall time: 5.99 s


MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

In [6]:
%%time
df_combined.head()

CPU times: user 2.4 ms, sys: 7.73 ms, total: 10.1 ms
Wall time: 64.9 ms


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


> Loading data and performing a simple EDA

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Adam Morphy | MacOS Big Sur | 8GB | 1.8 GHz Dual-Core Intel Core i5 | Yes | |
| Mukund Iyer | MacOS Monterey | 8GB | 1.4 GHz Quad-Core Intel Core i5 | Yes | |
| Julien Gordon | Ubuntu 20.04.4 LTS | 16GB | AMD® Ryzen 7 5800h with radeon graphics | Yes | |
| Shiva Shankar Jena | MacOS Catalina 10.15.7 | 4GB | 1.4 GHz Dual-Core Intel Core i5 | Yes | 3m 59s |

# 4.2 Approach 1 to reduce memory usage: Changing dtype

In [7]:
df_combined.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

In [8]:
print(f"Memory usage with float64: {df_combined[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df_combined[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2998.46 MB
Memory usage with float32: 1749.10 MB


In [20]:
%%time
df_combined_float32 = df_combined[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore')

CPU times: user 3.29 s, sys: 11.8 s, total: 15.1 s
Wall time: 1min 5s


## 4.3 Approach 2 to reduce memory usage: loading in chunks

In [17]:
%%time

# Doing EDA with only chunks of data
counts=pd.Series(dtype=int)
for chunk in pd.read_csv(
    "figsharerainfall/combined_data.csv",
    parse_dates=True,
    chunksize=1_000_000
):
    counts=counts.add(chunk.model.value_counts(), fill_value=0)

print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
CPU times: user 1min 28s, sys: 13.7 s, total: 1min 41s
Wall time: 1min 45s


# 5. Perform a simple EDA in R

## 5.1 Using parquet file to transfer the dataframe from python to R

In [18]:
# Using pandas
df_combined.to_parquet("figsharerainfall/combined_data_partition.parquet")

In [24]:
%load_ext rpy2.ipython

In [25]:
import rpy2_arrow.pyarrow_rarrow as pyra

In [26]:
%%R
suppressMessages(library(arrow, warn.conflicts = FALSE))
suppressMessages(library(dplyr, warn.conflicts = FALSE))

In [27]:
%%time
%%R
ds_rainfall <- open_dataset("figsharerainfall/combined_data_partition.parquet")

CPU times: user 18.5 ms, sys: 17.4 ms, total: 35.9 ms
Wall time: 40.3 ms


In [34]:
%%time
%%R
query <- ds_rainfall %>%
    select(model) %>%
    group_by(model) %>%
    summarise(
        count = n()
    )

CPU times: user 37.2 ms, sys: 19.4 ms, total: 56.6 ms
Wall time: 77.8 ms


In [36]:
%%time
%%R
print(query %>% collect())

# A tibble: 27 × 2
   model              count
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 FGOALS-f3-L      3219300
 4 CMCC-CM2-HR4     3541230
 5 MRI-ESM2-0       3037320
 6 GFDL-CM4         3219300
 7 BCC-CSM2-MR      3035340
 8 EC-Earth3-Veg-LR 3037320
 9 CMCC-ESM2        3541230
10 ACCESS-ESM1-5    1610700
# … with 17 more rows
CPU times: user 6.98 s, sys: 4.4 s, total: 11.4 s
Wall time: 9.31 s


> EDA time in R using parquet file --> arrow object ---> EDA ---> collect() (including loading time)

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Adam Morphy | MacOS Big Sur | 8GB | 1.8 GHz Dual-Core Intel Core i5 | Yes | |
| Mukund Iyer | MacOS Monterey | 8GB | 1.4 GHz Quad-Core Intel Core i5 | Yes | |
| Julien Gordon | Ubuntu 20.04.4 LTS | 16GB | AMD® Ryzen 7 5800h with radeon graphics | Yes | |
| Shiva Shankar Jena | MacOS Catalina 10.15.7 | 4GB | 1.4 GHz Dual-Core Intel Core i5 | Yes | 10s |